In [119]:
import time
from bs4 import BeautifulSoup
import pandas as pd
import os
import requests
import json
import re

In [120]:
# 사용자 에이전트
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:60.0) Gecko/20100101 Firefox/60.0',
    'Referer': 'https://www.musinsa.com/',
    'Accept-Language': 'en-US,en;q=0.5'
}

In [121]:
df = pd.read_csv('product_numbers.csv')['num']

In [160]:
df[:2]

0    2474966
1    3149296
Name: num, dtype: int64

In [123]:
def get_response(url, headers):

    # GET 요청
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')
    time.sleep(0.5)

    return soup


In [156]:
def extract_one_review(review, product_num):
    # 댓글 단 사람
    name = review.select('p.review-profile__name')[0].text
    
    # 메타 데이터
    meta_dict = {}
    metas = review.select('li.review-evaluation--type3__item')
    for meta in metas:
        key, value = meta.text.split(' ', 1)  # 공백을 기준으로 처음 나오는 부분만 분리
        meta_dict[key] = value

    # 리뷰 내용
    content = review.select('div.review-contents__text')[0].text

    # 배지
    badge = review.select('span.review-evaluation-button--type3__count')

    # 도움돼요
    helpful = badge[0].text

    # 스타일 좋아요
    try:
        style_good = badge[1].text
        
    except:
        style_good = ''

    data = {
        'product_num': product_num,
        'name': name,
        'meta_data': meta_dict,
        'content': content,
        'helpful': helpful,
        'style_good': style_good
    }

    return data

In [157]:
def extract_page_review_info(soup, product_num):
    one_page_reviews = []
    reviews = soup.select('div.review-list')
    for review in reviews:
        review_dict = extract_one_review(review, product_num)
        one_page_reviews.append(review_dict)
        
    return one_page_reviews


In [152]:
product_num = df[0]
all_reviews = []
page_num = 1
while True:
    url = f'https://goods.musinsa.com/api/goods/v2/review/style/list?similarNo=0&sort=up_cnt_desc&selectedSimilarNo={product_num}&page={page_num}&goodsNo={product_num}&rvck=202404150551&_=1713416512533'
    soup = get_response(url, headers)
    if not soup.select('p.review-profile__name'):
        break
    one_page_reviews = extract_page_review_info(soup, product_num)
    for one_page_review in one_page_reviews:
        all_reviews.append(one_page_review)
    page_num += 1

LV.4 뉴비_cbba72d7dfd2
LV.5 떵 주
LV.6 별콩아
LV.5 mrchef
LV.7 TOJIBF
LV.5 JuNe준
LV.3 B깐도
LV.5 한스브로
LV.4 qdqdq2
LV.6 패완얼이다이놈
LV.5 영쮸니
LV.7 강한_남자
LV.5 Freddd
LV.4 준스브
LV.5 귀여미라고해라
LV.5 호뤵
LV.5 Wsdga
LV.6 mon2347
LV.4 이구다
LV.6 폴워니
LV.5 오영흔
LV.5 qumber
LV.5 P0s1
LV.6 토트넘 홋스퍼 FC
LV.5 1벨
LV.6 주식안함
LV.5 Fdghgj
LV.5 Groenning
LV.5 203501
LV.6 마구로스시
LV.5 ioung99
LV.7 로리악
LV.4 wogurs
LV.5 뉴비_c9f436d2
LV.4 뉴비_f41a934f12d8
LV.5 돼지불백집사장
LV.5 내도신사
LV.5 창미니05
LV.5 시호찡
LV.6 MINOKIO
LV.3 MXTAPE
LV.3 마구창
LV.6 이코노미 진
LV.5 PILEN
LV.6 HHyeok_
LV.6 행크7
LV.5 건아아아
LV.5 pacing_pipe
LV.5 그라가스장
LV.5 나멩이
LV.6 작은근육고릴라
LV.3 뉴비_52a57d85
LV.4 멤버_dac9a9df623e
LV.6 갓성운
LV.5 뉴비_6a8fa46af0ca
LV.5 y0o0ung
LV.7 현승s
LV.6 권여붕
LV.7 삑카츄
LV.6 무채색좋아용
LV.5 ZzxxccV
LV.6 notbitc
LV.5 촘촘한초코칩
LV.3 뉴비_408b6d26
LV.4 gwx
LV.6 마이비호일
LV.3 픽시말리온
LV.6 개굴성민
LV.4 Remak
LV.5 ekc004
LV.5 55555555555555
LV.3 뉴비_d718c18a30bf
LV.7 참좋네유
LV.5 권차안
LV.5 꼬끄루
LV.3 멤버_511589367c2e
LV.5 CMXIV
LV.5 지훗
LV.6 토트넘 홋스퍼 FC
LV.3 승원우띠
LV.6 케이엠에이치
LV.5 Alisn
LV.7 십자상처
LV

In [161]:
def main():
    all_reviews = []
    for product_num in df[:2]:
        review_types = ['style', 'goods', 'photo']
        product_num = str(product_num)
        for review_type in review_types:
            page_num = 1

            while True:
                url = f'https://goods.musinsa.com/api/goods/v2/review/{review_type}/list?similarNo=0&sort=up_cnt_desc&selectedSimilarNo={product_num}&page={page_num}&goodsNo={product_num}&rvck=202404150551&_=1713416512533'
                soup = get_response(url, headers)
                
                if not soup.select('p.review-profile__name'):
                    break   

                one_page_reviews = extract_page_review_info(soup, product_num)
                for one_page_review in one_page_reviews:
                    all_reviews.append(one_page_review)
                
                page_num += 1


10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
283
293
303
313
323
333
343
353
363
373
383
393
403
413
423
433
443
453
463
473
483
493
503
513
523
533
543
553
563
573
583
593
603
613
623
633
643
653
663
673
683
693
703
713
723
733
743
753
763
773
783
793
803
813
823
833
843
853
863
873
883
893
903
913
923
933
943
953
963
973
983
993
1003
1013
1023
1033
1043
1053
1063
1073
1083
1093
1103
1113
1123
1133
1143
1153
1163
1173
1183
1193
1203
1213
1223
1233
1243
1253
1263
1273
1283
1293
1303
1313
1323
1333
1343
1353
1363
1373
1383
1393
1403
1413
1423
1433
1443
1453
1461
1471
1481
1491
1501
1511
1521
1531
1541
1551
1561
1571
1581
1591
1601
1611
1621
1631
1641
1651
1661
1671
1681
1691
1701
1711
1721
1731
1741
1751
1761
1771
1781
1791
1801
1805
1815
1825
1835
1845
1855
1865
1875
1885
1895
1905
1915
1925
1935
1945
1955
1965
1975
1985
1995
2005
2015
2025
2035
2045
2055
2065
2075
2085
2095
2105
2115
2125
2135
2145
2155
2165
2175
2185
2195
2205

In [162]:
pd.DataFrame(all_reviews)

,product_num,name,meta_data,content,helpful,style_good
0,2474966,LV.4 뉴비_cbba72d7dfd2,"{'사이즈': '보통이에요', '밝기': '어두워요', '색감': '선명해요', '...",꼭 사보시는걸 추천드립니다 실제로 사면 사진 보다 더 이쁘고 착용감이 예상보다 훨씬...,0,0
1,2474966,LV.5 떵 주,"{'사이즈': '커요', '밝기': '어두워요', '색감': '보통이에요', '두께...",한여름에 입어도 되겠다는 글을 봤어요.벌받았으면 좋겠어요. 근데 이쁘긴해요.,0,0
2,2474966,LV.6 별콩아,"{'사이즈': '보통이에요', '밝기': '보통이에요', '색감': '보통이에요',...",형이랑 같이 입으려고 또 구매했어용 이번엔 다른 색상도 구매했습니다 굿,0,0
3,2474966,LV.5 mrchef,"{'사이즈': '보통이에요', '밝기': '보통이에요', '색감': '보통이에요',...",원래 니트 까실까실해서 잘안입는데 촉감도 좋고 많이 얇진 않은데 시원하고 좋네요. ...,0,0
4,2474966,LV.7 TOJIBF,"{'사이즈': '보통이에요', '밝기': '보통이에요', '색감': '보통이에요',...",여름용 반팔니트로 딱이에요 가격도 적당해서 잘 산 것 같습니다,0,0
...,...,...,...,...,...,...
7038,3149296,LV.5 Style wani,"{'사이즈': '보통이에요', '밝기': '보통이에요', '색감': '보통이에요',...",색상도 화면이랑 비슷하구 핏도 이쁘게 잘나와요 세일 기간에 잘산듯 합니다,0,
7039,3149296,LV.3 공세동선교사,"{'사이즈': '보통이에요', '밝기': '보통이에요', '색감': '보통이에요',...",사진이 어떻게 보일진 모르겠지만 아주 만족합니다 한 벌 더 살 의향도 있네요,0,
7040,3149296,LV.7 y0dle,"{'사이즈': '보통이에요', '밝기': '보통이에요', '색감': '보통이에요',...",색감이 정말 이쁘고 촉감이 부드러워서 맨살에도 입을 수 있겠네요,0,
7041,3149296,LV.5 zh._.hyeong,"{'사이즈': '보통이에요', '밝기': '밝아요', '색감': '선명해요', '두...",색감이 너무 예뻐서 구매했는데 원하는 색상 그대로라 만족스럽습니더!,0,


In [ ]:
if __name__ == '__main__':
    main()